<a href="https://colab.research.google.com/github/MeenaRuwandi/INCS_870_Project_IDSforMinorAttacks/blob/main/incs870_team4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install imbalanced-learn
!pip install scikit-learn

In [2]:
!pip install gdown
import gdown

In [4]:
# Required Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
file_ids = [
    '1zOeCqtGZjAj_nSLe3W5MH5pDQM28bPde',  # Replace with actual file ID
    '1fagr2rKW8EN-Psc5UlD9BFfa0rRRF6i3',  # Replace with actual file ID
    '1WL00LFkCA2ylV8_8c6jm52bnEhGLcKsq',   # Replace with actual file ID
    '1xr_QBU3Ab42nW1ELl0NUydx4Rbx60TVR',
    '1MpqHsC5wQSu9CxlhJimz25EVcYubSMhu',
    '18mKfJHXinmfwPpaYNU2D6xHwq4dQuEuL',
    '1h1lMHytamf4Kc66NfAfMfBzDbmwM5Ub5',
    '11__JhC64_D4ezxjF8d0dPNRDJF-TY6Pq'
]


# Download each file
for i, file_id in enumerate(file_ids):
    url = f'https://drive.google.com/uc?id={file_id}'
    gdown.download(url, f'dataset_{i + 1}.csv', quiet=False)  # Saves as dataset_1.csv, dataset_2.csv, etc.

Downloading...
From (original): https://drive.google.com/uc?id=1zOeCqtGZjAj_nSLe3W5MH5pDQM28bPde
From (redirected): https://drive.google.com/uc?id=1zOeCqtGZjAj_nSLe3W5MH5pDQM28bPde&confirm=t&uuid=0a4fc9ea-9300-4e09-9760-aa01d40c5f05
To: /content/dataset_1.csv
100%|██████████| 225M/225M [00:04<00:00, 53.5MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1fagr2rKW8EN-Psc5UlD9BFfa0rRRF6i3
From (redirected): https://drive.google.com/uc?id=1fagr2rKW8EN-Psc5UlD9BFfa0rRRF6i3&confirm=t&uuid=4a6fd525-aef6-447a-b4a1-aef43ae642e5
To: /content/dataset_2.csv
100%|██████████| 135M/135M [00:02<00:00, 54.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1WL00LFkCA2ylV8_8c6jm52bnEhGLcKsq
To: /content/dataset_3.csv
100%|██████████| 52.0M/52.0M [00:00<00:00, 54.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1xr_QBU3Ab42nW1ELl0NUydx4Rbx60TVR
To: /content/dataset_4.csv
100%|██████████| 83.1M/83.1M [00:00<00:00, 100MB/s]
Downloading...
From (original): https://drive.g

In [6]:
dataset_paths = [
    '/sample_data/dataset_1.csv','/sample_data/dataset_2.csv','/sample_data/dataset_3.csv','/sample_data/dataset_4.csv',
    '/sample_data/dataset_5.csv','/sample_data/dataset_6.csv','/sample_data/dataset_7.csv','/sample_data/dataset_8.csv',
]

# Make the data frame
dataframes = [pd.read_csv(path) for path in dataset_paths]

FileNotFoundError: [Errno 2] No such file or directory: '/sample_data/dataset_1.csv'